<a href="https://colab.research.google.com/github/mightyoctopus/amazon-pricer-model-open-source-fine-tuned-models/blob/main/w7_d3_sft_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training with SFT Trainer

In [ ]:
# pip installs

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

In [2]:
import os
import re
import math

from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
from transformers.utils.import_utils import BASE_FILE_REQUIREMENTS
### Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "product-pricer"
HF_USER = "MightyOctopus"

### Data

# https://huggingface.co/datasets/MightyOctopus/amazon-pricer-dataset-v2-0
DATASET_NAME = f"{HF_USER}/amazon-pricer-dataset-v2-0"
MAX_SEQUENCE_LENGTH = 182

### Run name for saving the model in the hub:
RUN_NAME = f"{datetime.now():%Y-$m-%d_H%.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"


### Hyperparameters for QLoRA
LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True


### Hyperparameters for Training
EPOCHS = 1 # more than 1 might be overkill
BATCH_SIZE = 4 # Great for T4. If A100, this can be up to 16
GRADIENT_ACCUMULATION_STEPS = 8 # Better generalization and stable learning
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = "cosine"
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"


### Admin Config:
STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
HUB_MODEL_NAME